* ##### Exercise: Lottery Numbers
* ##### Name: Barath Anandaraman
* ##### Course: DSC640-T301
* ##### Weeks 7 & 8: Visualizing Relationships
* ##### Date: 05/03/2025

---

In [3]:
import pandas as pd

In [11]:
def preprocess_lottery_data(df, game_type, extra_ball=None):
    df = df.copy()
    # Extract Draw date
    df['Draw Date'] = pd.to_datetime(df['Draw Date'])

    # Split winning numbers into separate columns
    winning_numbers_split = df['Winning Numbers'].str.split(" ", expand=True)
    winning_numbers_split.columns = [f'Num{i+1}' for i in range(winning_numbers_split.shape[1])]
    df = pd.concat([df[['Draw Date']], winning_numbers_split], axis=1)

    # Add Mega/Power Ball if applicable
    if extra_ball and extra_ball in df.columns:
        df[extra_ball] = df[extra_ball].astype(str)
        #df = pd.concat([df, df[[extra_ball]].rename(columns={extra_ball: 'Extra_Ball'})], axis=1)
        df['Extra_Ball'] = df[extra_ball]
    else:
        df['Extra_Ball'] = None

    df['Game'] = game_type
    return df

##### Load datasets

In [4]:
mega_millions_df = pd.read_csv(r"Lottery_Mega_Millions_Winning_Numbers__Beginning_2002_20240421.csv")
powerball_df = pd.read_csv(r"Lottery_Powerball_Winning_Numbers__Beginning_2010_20240421.csv")
pick10_df = pd.read_csv(r"Lottery_Pick_10_Winning_Numbers__Beginning_1987_20240421.csv")

In [12]:
# Process each dataset
processed_mega = preprocess_lottery_data(mega_millions_df, 'Mega Millions', 'Mega Ball')
processed_powerball = preprocess_lottery_data(powerball_df, 'Powerball')
processed_pick10 = preprocess_lottery_data(pick10_df, 'Pick 10')

In [13]:
# Combine all datasets into one for Tableau
combined_df = pd.concat([processed_mega, processed_powerball, processed_pick10], ignore_index=True)

In [15]:
# Reshape data for Tableau
number_cols = [col for col in combined_df.columns if col.startswith("Num")]
melted_df = combined_df.melt(
    id_vars=["Draw Date", "Game", "Extra_Ball"],
    value_vars = number_cols,
    var_name="Position",
    value_name="Number"
)

In [16]:
# Remove nulls
melted_df = melted_df.dropna(subset=["Number"])

In [17]:
# Save to CSV for Tableau use
melted_df.to_csv('Reshaped_Lottery_Winning_Numbers.csv', index=False)

In [10]:
# Save to CSV for Tableau use
combined_df.to_csv('Combined_Lottery_Winning_Numbers.csv', index=False)